You might have heard about the term Gaussian boson sampling (GBS) when people discuss about quantum computing. GBS has often been used as a demonstration of *quantum supremacy*, a demonstration that a quantum computer can efficiently perform a computation which is intractable using classical computer, therefore challenging the extended Church-Turing thesis. GBS does not need a universal fault-tolerant quantum computer to be executed, instead it can be executed on a noisy intermediate scale quantum (NISQ) device.

Before going into GBS, let's stary with (just) Boson Sampling first. So, what is Boson Sampling?

Boson Sampling is a process of sampling the probability distribution of single photon measurements from the output of an interferometer in the following linear optical experiment: an array of single-photon sources is prepared to simultaneously emit single photon states into a multimode linear interferometer. Why is this a classically intractable problem?

Consider $N$-dimensional single photon Fock states, $|\psi\rangle=\left|m_{1}, m_{2}, \ldots, m_{N}\right\rangle$, composed of $b=\sum_{i} m_{i}$ photons, incident on an $N$-mode linear interferometer, which performs the following linear transformation of the input mode creation and annihilation operators:
$$
\begin{aligned}
&\hat{a}_{o u t_{k}}^{\dagger}=\sum_{j=0}^{N} U_{k j} \hat{a}_{i n_{j}}^{\dagger} \\
&\hat{a}_{o u t_{k}}=\sum_{j=0}^{N} U_{j k}^{\dagger} \hat{a}_{i n_{j}}
\end{aligned}
$$
Here, the unitary $U$ describes the quantum evolution or the action of the interferometer on the quantum state. The probability of detecting $n_{j}$ photons at the $j$ th output mode is given by
$$
\left|\left\langle n_{1}, n_{2}, \ldots, n_{N}|W| \psi\right\rangle\right|^{2}
$$
where $W$ is a homomorphism of $U$ which encodes the transformation of the quantum state in the Fock basis. The single photon probability distribution at the output of the interferometer can be shown to be
$$
\left|\left\langle n_{1}, n_{2}, \ldots, n_{N}|W| \psi\right\rangle\right|^{2}=\frac{\left|\operatorname{Per}\left(U_{s t}\right)\right|^{2}}{m_{1} ! m_{2} ! \cdots m_{N} ! n_{1} ! n_{2} ! \cdots n_{N} !}
$$
where $\operatorname{Per}\left(U_{s t}\right)$ is the permanent of $U_{s t}$, a submatrix of the interferometer unitary.

The permanent of a matrix, defined by
$$
\operatorname{Per}(A)=\sum_{\sigma=S_{N}} \prod_{i=1}^{N} A_{i \sigma(i)}
$$
where $S_{N}$ is the set of all permutations of $N$ elements, is calculated in a similar fashion to the determinant, but unlike the determinant, the signatures of the permutations are not taken into account-every permutation is taken as a positive quantity.
In graph theory, the permanent calculates the number of perfect matchings in a bipartite graph with adjacency matrix $A$.

The determinant of a matrix can be computed in polynomial time in classical computer. However, computing the permanent of a matrix belongs to #P-hard computational complexity class, which contains all NP-complete problems. Not only that, even calculating the permanent in an approximate manner is a member of #P-hard.

Does this means that our quantum computer can solve NP-complete problems efficiently? The answer is no. The algorithm we just presented only sample a distribution whose probabilities coincide with the square of the permanent of submatrices of the unitary representing a linear interferometer.

A note on computing permanent. If the notation above seems rather cryptic, you can check your understanding with an explicit example below.

$\operatorname{perm}\left(\begin{array}{lll}a & b & c \\ d & e & f \\ g & h & i\end{array}\right)=a e i+b f g+c d h+c e g+b d i+a f h$

Not too difficult isn't it? Now, we will try to do a Boson sampling simulation in Strawberry Fields. Here is the circuit that we are going to implement.

In [1]:
import numpy as np

# set the random seed
np.random.seed(42)

# import Strawberry Fields
import strawberryfields as sf
from strawberryfields.ops import *

# initialize a 4 mode program
boson_sampling = sf.Program(4)

with boson_sampling.context as q:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac     | q[2]
    Fock(1) | q[3]

    # rotation gates
    Rgate(0.5719)  | q[0]
    Rgate(-1.9782) | q[1]
    Rgate(2.0603)  | q[2]
    Rgate(0.0644)  | q[3]

    # beamsplitter array
    BSgate(0.7804, 0.8578)  | (q[0], q[1])
    BSgate(0.06406, 0.5165) | (q[2], q[3])
    BSgate(0.473, 0.1176)   | (q[1], q[2])
    BSgate(0.563, 0.1517)   | (q[0], q[1])
    BSgate(0.1323, 0.9946)  | (q[2], q[3])
    BSgate(0.311, 0.3231)   | (q[1], q[2])
    BSgate(0.4348, 0.0798)  | (q[0], q[1])
    BSgate(0.4368, 0.6157)  | (q[2], q[3])

In [2]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 10})
results = eng.run(boson_sampling)
probs = results.state.all_fock_probs()

In [3]:
print(probs[1, 1, 0, 1])
print(probs[2, 0, 0, 1])

0.17468916048563934
0.10644192724642344


Now, what is a Gaussian Boson Sampling (GBS)?

The Gaussian boson sampling scheme remains, on initial observation, quite similar to that of boson sampling:
- $N$ single mode squeezed states $|z\rangle$, with squeezing parameter $z=r e^{i \phi}$, enter an $N$ mode linear interferometer described by unitary $U$ simultaneously.
- Each output mode of the interferometer (denoted by state $\left|\psi^{\prime}\right\rangle$ ) is then measured in the Fock basis, $\bigotimes_{i} n_{i}\left|n_{i}\right\rangle\left\langle n_{i}\right|$
Without loss of generality, we can absorb the squeezing phase parameter $\phi$ into the interferometer, and set $\phi=0$ for convenience.
Using phase space methods, Hamilton et al. [2] showed that the probability of measuring a Fock state containing only 0 or 1 photons per mode is given by
$$
\left|\left\langle n_{1}, n_{2}, \ldots, n_{N} \mid \psi^{\prime}\right\rangle\right|^{2}=\frac{\left|\operatorname{Haf}\left[\left(U\left(\bigoplus_{i} \tanh \left(r_{i}\right)\right) U^{T}\right)\right]_{s t}\right|^{2}}{\prod_{i=1}^{N} \cosh \left(r_{i}\right)}
$$
i.e., the sampled single photon probability distribution is proportional to the hafnian of a submatrix of $U\left(\bigoplus_{i} \tanh \left(r_{i}\right)\right) U^{T}$, dependent upon the output covariance matrix.

The hafnian of a matrix is defined by
$$
\operatorname{Haf}(A)=\frac{1}{n ! 2^{n}} \sum_{\sigma \in S_{2 N}} \prod_{i=1}^{N} A_{\sigma(2 i-1) \sigma(2 i)}
$$
where $S_{2 N}$ is the set of all permutations of $2 N$ elements. In graph theory, the hafnian calculates the number of perfect matchings in an arbitrary graph with adjacency matrix $A$.
Compare this to the permanent, which calculates the number of perfect matchings on a bipartite graph - the hafnian turns out to be a generalization of the permanent, with the relationship
$$
\operatorname{Per}(\mathrm{A})=\operatorname{Haf}\left(\left[\begin{array}{cc}
0 & A \\
A^{T} & 0
\end{array}\right]\right)
$$

It follows that computing a Hafnian is a #P-hard problem.

In [4]:
# define the linear interferometer
U = np.array([
 [ 0.219546940711-0.256534554457j, 0.611076853957+0.524178937791j,
    -0.102700187435+0.474478834685j,-0.027250232925+0.03729094623j],
 [ 0.451281863394+0.602582912475j, 0.456952590016+0.01230749109j,
    0.131625867435-0.450417744715j, 0.035283194078-0.053244267184j],
 [ 0.038710094355+0.492715562066j,-0.019212744068-0.321842852355j,
    -0.240776471286+0.524432833034j,-0.458388143039+0.329633367819j],
 [-0.156619083736+0.224568570065j, 0.109992223305-0.163750223027j,
    -0.421179844245+0.183644837982j, 0.818769184612+0.068015658737j]
])

In [5]:
# create the 4 mode Strawberry Fields program
gbs = sf.Program(4)

with gbs.context as q:
    # prepare the input squeezed states
    S = Sgate(1)
    S | q[0]
    S | q[1]
    S | q[2]
    S | q[3]

    # linear interferometer
    Interferometer(U) | q

In [6]:
eng = sf.Engine(backend="gaussian")
results = eng.run(gbs)

In [7]:
# Fock states to measure at output
measure_states = [[0,0,0,0], [1,1,0,0], [0,1,0,1], [1,1,1,1], [2,0,0,0]]

# extract the probabilities of calculating several
# different Fock states at the output, and print them to the terminal
for i in measure_states:
    prob = results.state.fock_prob(i)
    print("|{}>: {}".format("".join(str(j) for j in i), prob))

|0000>: 0.17637844761413482
|1100>: 0.06855956371224513
|0101>: 0.002056097258972289
|1111>: 0.008342946399881963
|2000>: 0.0103129452534403
